# Greening Research In Tacoma
## August Grit City Temperature Blitz Data Cleaning and Standardization
This following data was collected on August 11th, 2022 over the course of 3 "shifts" of data collection by volunteers. Each volunteer took a temperature logger (with no solar radiation shield) to specified electrical poles in Tacoma that had their own temperature logger (equipped with a radiation shield) previously installed. The temperature loggers on the electrical poles were mounted 8' from the ground while the sensors carried with the volunteers were placed on the ground for 5 minutes for their temperature to stabilize. 

Each location with an installed temperature logger on a pole has a myriad of recorded data tied to it. This data includes: exact location/elevation, number of trees surrounding and their location, its matching electrical pole number, a specific frequency of temperature recordings (minute/hour), data of installation, and more. 

The purpose of this research is to quantify the efficacy of tree canopy coverage in an urban environment. For this purpose, temperature loggers have been installed in locations covering a complete spectrum of tree coverage: from heavy coverage to no coverage whatsoever. The temperature loggers on the surface were used to ascertain the difference between temperatures on the ground and 8' above the ground. 

### Importing of Necessary Libraries

In [50]:
###import Libraries
import pandas as pd
import numpy as np
from datetime import timedelta
from datetime import datetime
import datetime as dt
from cmath import pi

### Initial Standardization
Start and end times are standardized in this code block. These times were manually entered by a variety of volunteers and included formats such as 1:34, 1:34pm, 13:34, or, in some cases, a completely incorrectly input value (which is handled separately). This code strips these inputs down to 4 digit numbers (with a 0 in front if necessary) and returns completely standardized 24hr values for each logged start/end times. It then adds an additional DateTime value column for start and end times.

In [51]:
#read file
Blitz = pd.read_csv('TempBlitzFormData.csv')
Blitz.dropna(subset=['Start time (hh:mm AM/PM)', 'End time (hh:mm AM/PM)'], inplace=True) #drop null time inputs

def time_sanitation(t):
    ###Standardizes user-input times###
    t = t.str.replace('AM|PM|am|pm|:|;', '', regex=True).str.strip().astype(int) #strips input to numbers
    
    standardization = []
    for x in t: 
        if x < 100: standardization.append(x*100) #change all values such as 12 to 1200
        if x >= 100 and x < 1299: standardization.append(x) #keep all values in standard 12hr time
        if x > 1299: standardization.append(x-1200) #change all military times to 12hr
    t = standardization
    
    length = []
    for x in t: 
        
        if len(str(x)) < 4: length.append(f'0{x}') #if 3 digit number, add 0 in front of value (eg 134 to 0134)
        else: length.append(x)
    t = length
    
    to_time = []    
    for x in t:
        
        x = pd.to_datetime(x, format='%H%M') #returns 1900-01-01 xx:xx:xx
        y = x.time() #remove placehoder dates, keeps time formatting
        to_time.append(y)    
    t = to_time
    ampm = []
    for x in t: #appends AM or PM based on input time
        if x.hour > 7 and x.hour < 12: ampm.append(f'{x} AM')
        if x.hour < 7 or x.hour >= 12: ampm.append(f'{x} PM')
    t = ampm
    
    return t

Blitz['Start time (hh:mm AM/PM)'] = time_sanitation(Blitz['Start time (hh:mm AM/PM)'])
Blitz['End time (hh:mm AM/PM)'] = time_sanitation(Blitz['End time (hh:mm AM/PM)'])

#single end time value was input as 12:37 instead of 2:37. 2:37 matches 2:32 start time and 15:35 Timestamp
Blitz.loc[86, 'End time (hh:mm AM/PM)'] = '02:37:00 PM'

#single start time value was input as 5:57 instead of 15:57. 2:57 matches 3:02 end time and 16:02 Timestamp
Blitz.loc[108, 'Start time (hh:mm AM/PM)'] = '02:57:00 PM'

###to remove "practice" values prior to August:
#remove timezone from Timestamp so that it can be read as a DateTime
Blitz['Timestamp'] = Blitz['Timestamp'].str.replace('MDT', '', regex=True)
#extract month from DateTime
Blitz['Month'] = pd.DatetimeIndex(Blitz['Timestamp']).month
#ignoring dates not part of August blitz - not sure if these are needed
AugBlitz = Blitz[Blitz['Month'] == 8].reset_index(drop=True)

###Added 24hr start and end time columns
def to_24hr(t):
    ####Converts 00:00:00 AM/PM times to 24 hour formatting
    military_times = []
    for x in t:
        in_time = datetime.strptime(x, "%I:%M:%S %p")
        out_time = datetime.strftime(in_time, "%H:%M:%S")
        military_times.append(out_time)
    t = military_times
    return t
AugBlitz['Start time (hh:mm:ss)'] = to_24hr(AugBlitz['Start time (hh:mm AM/PM)']) #returns new 24hr Start Time column
AugBlitz['End time (hh:mm:ss)'] = to_24hr(AugBlitz['End time (hh:mm AM/PM)']) #returns new 24 End Time column


AugBlitz['EndTime DateTime'] = pd.to_datetime(AugBlitz['End time (hh:mm:ss)']) #converts 24 hour format to DateTime
AugBlitz['EndTime DateTime'] = AugBlitz['EndTime DateTime'].apply(lambda dt: dt.replace(year=2022, month=8, day=11)) #sets correct date

EndTimeList = []
for time in AugBlitz['EndTime DateTime']:
    EndTimeList.append(time.replace(second=0, microsecond=0, minute=time.minute, hour=time.hour)+timedelta(minutes=time.second//30))
AugBlitz['EndTime'] = EndTimeList
AugBlitz['EndTime'] = AugBlitz['EndTime'].dt.strftime('%H:%M') #converts format that matches logger data

AugBlitz['StartTime DateTime'] = pd.to_datetime(AugBlitz['Start time (hh:mm:ss)']) #converts 24 hour format to DateTime
AugBlitz['StartTime DateTime'] = AugBlitz['StartTime DateTime'].apply(lambda dt: dt.replace(year=2022, month=8, day=11)) #sets correct date

StartTimeList = []
for time in AugBlitz['StartTime DateTime']:
    StartTimeList.append(time.replace(second=0, microsecond=0, minute=time.minute, hour=time.hour)+timedelta(minutes=time.second//30))
AugBlitz['StartTime'] = StartTimeList
AugBlitz['StartTime'] = AugBlitz['StartTime'].dt.strftime('%H:%M') #converts to matching format


### Logger Number Correction
This code standardizes and corrects user input Pole #s and Pole Logger #s. There were a large amount of incorrectly input data here as data collectors manually typed large and unfamiliar numbers. This data needed to be corrected and was cross referenced with other known values - for instance, if a pole logger number was mistyped, the pole logger number can be found and checked by looking up the pole number. If the discrepency was a single miss-typed character, that value is 

In [52]:
#correcting user input logger # values
logger = []
for x in AugBlitz['Your Temperature Logger #']:
    if len(x) > 8: #several results were formatted MX2201 xxxxxxxx. Kept only last 8 digits
        logger.append(x[-8:])
    if len(x) == 8 and x != '21402965': #8 digit values are presumed correct for now. 21402965 is not in a valid range
        logger.append(x)
    if len(x) < 8 or x == '21402965': #index #109 was found mistyped as 2130965. Cross-referenced user's other values.
        logger.append('21302965')     #index #78 was found mistyped as 21402965. Cross-referenced user's other values.
AugBlitz['Your Temperature Logger #'] = logger #following changes, all loggers found correctly grouped

###Standardizing and cleaning pole and logger #s
###values cross-referenced with HoboLocations_PoleLocations.csv (referred to as HL_PL.csv)

AugBlitz['Utility Pole #'] = AugBlitz['Utility Pole #'].str.upper() #standardized capitalization
AugBlitz['Utility Pole Temperature Logger # '] = AugBlitz['Utility Pole Temperature Logger # '].str.upper()
AugBlitz['Utility Pole #'] = AugBlitz['Utility Pole #'].str.replace('TO','TP') #Corrected pole # typo
AugBlitz['Utility Pole Temperature Logger # '] = AugBlitz['Utility Pole Temperature Logger # '].str.replace('B7','BT') #Corrected pole # typo
AugBlitz['Utility Pole Temperature Logger # '] = AugBlitz['Utility Pole Temperature Logger # '].str.replace('B1','BT') 

AugBlitz.loc[[38, 79], 'Utility Pole #'] = 'CP10'                 #no visible number; collaborated locations with and standardized to HL_PL.csv
AugBlitz.loc[[85, 9, 32], 'Utility Pole #'] = 'NO NUM'            #no pole num; collaborated locations with and standardized to HL_PL.csv
AugBlitz.loc[[33], 'Utility Pole #'] = 'TPU927'                   #pole number not listed. times and logger match. index 83 notes collaborate this
AugBlitz.loc[[6], 'Utility Pole #'] = 'TP24408'                   #minor typo: TP254408 input. TP24408 matches pole/ logger number in HL_PL.csv
AugBlitz.loc[[22], 'Utility Pole #'] = 'TP24199'                  #minor typo: TP2499 input. TP24199 matches pole/ logger number in HL_PL.csv
AugBlitz.loc[[86], 'Utility Pole #'] = 'TP3890'                   #minor typo: TP3800 input. TP3890 matches pole/ logger number in HL_PL.csv
AugBlitz.loc[[83], 'Utility Pole #'] = 'TPU927'                   #minor typo: TPU827 input. TPU927 matches pole/ logger number in HL_PL.csv
AugBlitz.loc[[54], 'Utility Pole #'] = 'TP6340'                   #minor typo: TP 6340 input. TP6340 matches pole/ logger number in HL_PL.csv
AugBlitz.loc[[93, 25], 'Utility Pole #'] = 'TP21897/A1353217'     #minor typo on #93: TP21879 input. TP21897 matches pole/ logger number in HL_PL.csv 
                                                                  #Standardized both to HL_PL.csv
AugBlitz.loc[[23, 72, 47], 'Utility Pole #'] = '52/1'             #standardized pole number to HL_PL.csv
AugBlitz.loc[[77, 81, 27], 'Utility Pole #'] = 'TP21631/A1353441' #standardized pole number to HL_PL.csv
AugBlitz.loc[[18], 'Utility Pole #'] = 'TP21374'                  #standardized pole number to HL_PL.csv
AugBlitz.loc[[97, 53], 'Utility Pole #'] = 'TP32711/A0572122'     #standardized pole number to HL_PL.csv
AugBlitz.loc[[11, 108], 'Utility Pole #'] = 'TP25239/A1353202'    #standardized Pole #s. Likely typo on HL_PL.csv: input value BT2130297 should be BT21302971
AugBlitz.loc[[26], 'Utility Pole #'] = 'TP21629'                  #standardized Pole #s. Likely typo in HL_PL.csv: input value BT2130251 should be BT21302951
AugBlitz.loc[[20, 68, 55], 'Utility Pole #'] = '112049 50/3'      #standardized pole #s. Likely typo in HL_PL.csv: input value 11049   50/3 should be 112049 50/3

AugBlitz.loc[[103], 'Utility Pole Temperature Logger # '] = '21223124'      #minor typo: 2122324 input. 21223124 matches pole/ logger number in HL_PL.csv
AugBlitz.loc[[3], 'Utility Pole Temperature Logger # '] = '21223121'        #minor typo: 211223121 input. 21223121 matches pole/ logger number in HL_PL.csv
AugBlitz.loc[[105], 'Utility Pole Temperature Logger # '] = '21223101'      #minor typo: 2122301 was input. 21223101 matches pole/ logger number in HL_PL.csv
AugBlitz.loc[[21], 'Utility Pole Temperature Logger # '] = '21223136'       #minor typo: 12236136 was input. 21223136 matches pole/ logger number in HL_PL.csv
AugBlitz.loc[[74], 'Utility Pole Temperature Logger # '] = '21223129'       #minor typo: 2122329 was input. 21223129 matches pole/ logger number in HL_PL.csv
AugBlitz.loc[[89], 'Utility Pole Temperature Logger # '] = '21223116'       #minor typo: 2122316 was input. 21223116 matches pole/ logger number in HL_PL.csv
AugBlitz.loc[[101], 'Utility Pole Temperature Logger # '] = 'BT21302953'    #minor typo: 21302953 was input. BT21302953 matches pole/ logger number in HL_PL.csv
AugBlitz.loc[[29, 0], 'Utility Pole Temperature Logger # '] = '21223116/BT21302967'      #standardized logger numbers to HL_PL.csv
AugBlitz.loc[[44, 12], 'Utility Pole Temperature Logger # '] = '2401795/BT21302957'      #standardized logger numbers to HL_PL.csv
AugBlitz.loc[[5, 35, 95], 'Utility Pole Temperature Logger # '] = '21162467/BT21302947'  #minor typo on #5: logger #21302954 was input. BT21302947 matches pole/ logger number in HL_PL.csv
                                                                                         #Standardized each logger number to HL_PL.csv
AugBlitz.loc[[36, 66, 98], 'Utility Pole Temperature Logger # '] = '21223140/BT21302974' #36 has a matching pole # with 66 and 98, but different logger #
                                                                                         #probable typo between HL_PL.csv value BT21302974 and recorded BT21202973
                                                                                         #logger conditions do not match (soil vs grass) and times are pretty close, but likely the same value
                                                                                         #standardized all 3 values
            
AugBlitz['Utility Pole Temperature Logger # '] = AugBlitz['Utility Pole Temperature Logger # '].str[-8:]

#index 24 and 87 both have Logger # BT21302966, but different Pole #s. 87 Matches HL_PL.csv
#HL_PL.csv has the same Hobo_SN for pole TP6335 and TP6325. Not sure how to label index 107. 107 also has no matching logger #
#84 has Pole # and Logger # matching HL_PL.csv, but no other matches.
#105 has Pole # and Logger # matching HL_PL.csv, but no other matches.
#46 has Pole # and Logger # matching HL_PL.csv, but no other matches.
#78 has Pole # and Logger # matching HL_PL.csv, but no other matches.
#18 has Pole # and Logger # matching HL_PL.csv, but no other matches.
#74 has Pole # and Logger # matching HL_PL.csv, but no other matches.
#89 has Pole # and Logger # matching HL_PL.csv, but no other matches.
#34 pole # TP10504 has no matching pole in Blitz data or HL_PL.csv. Logger # 21223129 matches for pole # A1341128 in HL_PL.CV

### Importing Surface Logger Data
Here the database of recorded temperature values of each surface logger is imported and standardized. Then a 'Time' column is added from the DateTime index that allows matching to the main database. It uses this information to scan the primary database and, using both surface logger ID number and time stamps, returns the matching start time temperature and end time temperature to the primary database in two new columns.

In [53]:
SurfaceLoggers = pd.read_csv('BlitzSurfaceLoggers.csv', skiprows=1, names=["21302948", "21302943", "21302954", "21302958", "21302965", "21302973", "21302978"])
SurfaceLoggers['21302949'] = 0 #unable to find data for this logger
SurfaceLoggers['21302945'] = 0 #unable to find data for this logger

time = [] 
for datetime in SurfaceLoggers.index: 
    time.append((datetime[-5:]).strip()) #extract only time from datetime and strip leading space if 1 digit hour and append to empty list
SurfaceLoggers['Time'] = time #save list as new column

time_standardization = []
for times in SurfaceLoggers['Time']: 
    if len(str(times)) < 5: time_standardization.append(f'0{times}') #add a leading zero if one digit hour
    else: time_standardization.append(times)
SurfaceLoggers['Time'] = time_standardization #overwrite Time column with leading zeros
SurfaceLoggers.to_csv(r'C:\Users\johnd\Desktop\Data Stuffs\GRIT City Temp Blitz\SurfaceLoggerValues')

####Merge surface and pole temperature logger information to AugBlitz

StartSurfaceTemp = []
#returns corresponding surface logger temperature for each time/logger combination in AugBlitz
for index, row in AugBlitz.iterrows():
    logger = row['Your Temperature Logger #']
    time = row['StartTime']
    for index, row in SurfaceLoggers.iterrows():
        if row['Time'] == time:
            StartSurfaceTemp.append(SurfaceLoggers[f'{logger}'].loc[index])
            break
AugBlitz['StartSurfaceTemp'] = StartSurfaceTemp

EndSurfaceTemp = []
#returns corresponding surface logger temperature for each time/logger combination in AugBlitz
for index, row in AugBlitz.iterrows():
    logger = row['Your Temperature Logger #']
    time = row['EndTime']
    for index, row in SurfaceLoggers.iterrows():
        if row['Time'] == time:
            EndSurfaceTemp.append(SurfaceLoggers[f'{logger}'].loc[index])
            break
AugBlitz['EndSurfaceTemp'] = EndSurfaceTemp

### Importing and Matching Pole Logger Data
This code accomplishes the same basic function as the surface logger section, but using a completely different method. While the surface logger data encompassed one day and contained all temperature data for every surface logger in one csv file, each pole logger has its own CSV file - some with 6+ months of data. Additionally, while the surface loggers each returned values minute by minute, some pole loggers return data per hour while others return data per minute. This code trims the data by using only data from the day of the Blitz and then standardizes the columns while dropping unused columns. This takes quite a while due to the massive amount of data. It then concatenates the data into one large database. From there, a merge_asof function is used to match the nearest value in the pole logger database to the relevant time in the main database and a column is added to reflect that matched data.

In [54]:
PoleLoggers = pd.DataFrame()
PoleLoggerNums = AugBlitz['Utility Pole Temperature Logger # '].unique().tolist()
#loads all applicable pole logger CSVs, standardizes everything, keeps only relevant times, and concats into a single DataFrame
for poleNumber in PoleLoggerNums:
    TempDF = pd.read_csv(f'Pole Temp CSVs/{poleNumber}.csv', encoding = "ISO-8859-1", skiprows = [0])
    TempDF = TempDF.iloc[:, [1,2]] #only interested in timestamp and temperature value - other columns are arbitrary or empty
    TempDF.columns.values[1] = 'Temp' #standardize columns
    TempDF.columns.values[0] = 'DateTime'
    TempDF['DateTime'] =  pd.to_datetime(TempDF['DateTime']) #change to datetime so date can be filtered
    TempDF['LoggerNum'] = poleNumber #include pole number in individual Dataframes so it is searchable when concat-ed
    TempDF = TempDF[(TempDF['DateTime'] > '2022-08-11 08:30:00') & (TempDF['DateTime'] < '2022-08-11 15:30:00')] #keep only times in usable range
    TempDF = TempDF.reset_index(drop=True) #begin index at 8:30am
    PoleLoggers = pd.concat([PoleLoggers, TempDF]) #add DF to combined DF of all polelogger CSVs

#Standardize newly created PoleLoggers Dataframe and sort by DateTime index
PoleLoggers = PoleLoggers.sort_values('DateTime')
PoleLoggers = PoleLoggers.set_index('DateTime')
PoleLoggers.index = pd.to_datetime(PoleLoggers.index)
PoleLoggers = PoleLoggers.rename(columns= {'LoggerNum': 'PoleLogger', 'Temp':'PoleTemp'})

#Standardize AugBlitz Dataframe and sort by DateTime index
AugBlitz = AugBlitz.rename(columns= {'End time (hh:mm AM/PM)':'EndTime', 'Start time (hh:mm AM/PM)':'StartTime'\
    , 'Your temperature logger is located in:':'Sun/Shade','Your Temperature Logger #': 'SurfaceLogger'\
    , 'Utility Pole Temperature Logger # ':'PoleLogger', 'EndTime DateTime': 'DateTime'})
AugBlitz.DateTime = pd.to_datetime(AugBlitz.DateTime)
AugBlitz = AugBlitz.sort_values('DateTime')
AugBlitz = AugBlitz.set_index('DateTime')

#merge AugBlitz and PoleLoggers by nearest times where logger matches.
AugBlitzValues = pd.merge_asof(AugBlitz, PoleLoggers, by='PoleLogger', direction='nearest', left_index = True, right_index= True)

#keep only useful columns and return to numeric index 
AugBlitzValues = AugBlitzValues.reset_index()
AugBlitzValues = AugBlitzValues.iloc[:, [0, 3, 6, 7, 8, 9, 15, 17, 18, 19, 20]]

### Adding Location Data
This next section adds locational data for each of the of the locations where data was collected. This data includes latitude and longitude values as well as a Y/N value for tree coverage. These are matched to the main dataset using the pole logger #s.

In [55]:
###loaded location/tree data for loggers to merge to AugBlitzValues
HoboLocations = pd.read_csv('HoboLocations_PoleLocations.csv')
HoboLocations = HoboLocations.iloc[:, [0,1,8,9,11]]
HoboLocations = HoboLocations[HoboLocations.Latitude < 47.5] #ignore traps from Seattle

###minor Hobo_locations corrections
#No matches found for 21223119, 21302963. 
#21302966 is a match from HoboLocations, but its corrdinates are in seattle
#21302967 exists twice in HoboLocations
HoboLocations.loc[[62], 'Hobo_SN'] = '21302971' #Originally 2130297
HoboLocations.loc[[57], 'Hobo_SN'] = '21302951' #Originally 2130951
HoboLocations.loc[[65], 'Hobo_SN'] = 'unknown1' #Originally 21302967, but this number is on HoboLocations twice and the pole doesn't match
                                                #the values collected during the blitz
HoboLocations.Hobo_SN = HoboLocations.Hobo_SN.str[-8:] #remove BTs

#match pole column with AugBlitzValue
HoboLocations = HoboLocations.rename(columns= {'Hobo_SN': 'PoleLogger'})
HoboLocations.PoleLogger.astype(str)
AugBlitzValues.PoleLogger = AugBlitzValues.PoleLogger.astype(str) #ensure same data type
AugBlitzNew = AugBlitzValues.merge(HoboLocations, how ='left', on= 'PoleLogger') #merge DFs together

### Calculating Tree Coverage
Using a new dataset that includes tree data (size of tree and distance from logger), a Basal Area calculation is added by using tree Diameter at Breast Height. The DBH is measured in centimeters and Basal Area is traditionally measured in meters. To calculated BA, the formula is the same as finding the area of a circle (pi \* radius squared) so to calculate this, the formula is pi\*(DBH_cm / 200)^2. Then each tree's Basal Area is summed together with other trees near the logging device.

An attempt to factor tree distance was made by using a distance decay formula. This formula multiplies the Basal Area by a Constant (currently unkown and substituted with 1) and then multiplies this number by the minus 2 exponent of the distance the tree is from the logger. With this method, the further the tree is from the logger, the exponentially smaller the number distance factor will be. This is expressed with the formula BA\*C\*(Distance^-2).

Both summed basal area and summed distance decay are then added as columns to the main database by matching to the pole logger number. 

In [56]:
AugBlitzNew.loc[[9, 42, 80], 'WptNo'] = 35
AugBlitzNew['WptNo'] = pd.to_numeric(AugBlitzNew['WptNo'])

TreeData = pd.read_csv('HoboLocations_TreeData.csv')
TreeData = TreeData.iloc[:, :7]
TreeData.loc[99, 'DBH_cm'] = 9
TreeData.loc[100, 'DBH_cm'] = 6
TreeData.loc[63, 'DistFromLogger_m'] = 5

TreeData['DBH_cm'] = pd.to_numeric(TreeData['DBH_cm'])
TreeData['DistFromLogger_m'] = pd.to_numeric(TreeData['DistFromLogger_m'])
TreeData = TreeData.dropna(subset=['DBH_cm', 'DistFromLogger_m'], how='all')

C = 1  #interference constant from distance decay formula. Constant currently unknown and substituted with 1.
TreeData['BA'] = pi*((TreeData['DBH_cm']/200)**2)
TreeData['DistanceDecay'] = TreeData['BA'] * C * (TreeData['DistFromLogger_m']**-2)

GroupedBasalArea = TreeData.groupby('WptNo')['BA'].sum().to_frame(name = 'BA').reset_index()
WeightedDistance = TreeData.groupby('WptNo')['DistanceDecay'].sum().to_frame(name = 'DistanceDecay').reset_index()
TreeEffects = GroupedBasalArea.merge(WeightedDistance, how ='inner', on= 'WptNo')
AugBlitzComplete = AugBlitzNew.merge(TreeEffects, how ='left', on= 'WptNo')
AugBlitzComplete['SummedBA'] = AugBlitzComplete['BA'].fillna(value=0)
AugBlitzComplete['DistanceDecay'] = AugBlitzComplete['DistanceDecay'].fillna(value=0)

### Final Dataset is Re-ordered and Presented
This code simply re-orders the columns and keeps only the relevant data that we want to analyze. It then saves this new database as a CSV file that can be referenced for analysis. The first 20 rows have been displayed to show the results.

In [57]:
AugBlitzComplete = AugBlitzComplete[['DateTime','StartTime', 'EndTime', 'SurfaceLogger', 'StartSurfaceTemp', 'EndSurfaceTemp', 'PoleLogger'\
                                    , 'PoleTemp', 'Utility Pole #', 'Latitude', 'Longitude', 'Sun/Shade', 'Trees?', 'SummedBA', 'DistanceDecay']]
AugBlitzComplete.to_csv('AugBlitzComplete.csv')
AugBlitzComplete.head(20)

,DateTime,StartTime,EndTime,SurfaceLogger,StartSurfaceTemp,EndSurfaceTemp,PoleLogger,PoleTemp,Utility Pole #,Latitude,Longitude,Sun/Shade,Trees?,SummedBA,DistanceDecay
0,2022-08-11 08:46:00,08:40,08:46,21302958,24.24,20.16,21302967,17.930,TP6262,47.213277,-122.483526,Shade,Y,2.424760,0.041437
1,2022-08-11 08:56:00,08:51,08:56,21302954,22.73,28.48,21302950,18.530,TP6340,47.216117,-122.473789,Sun,N,0.000000,0.000000
2,2022-08-11 08:56:00,08:50,08:56,21302958,20.93,19.30,21223106,18.711,TPU927,47.214566,-122.483749,Shade,N,0.000000,0.000000
3,2022-08-11 08:57:00,08:52,08:57,21302978,28.53,32.64,21223121,19.282,TP23269,47.216736,-122.480436,Sun,N,0.018928,0.000386
4,2022-08-11 09:00:00,08:55,09:00,21302973,24.66,30.07,21223123,19.187,TP20171,47.209033,-122.475016,Sun,N,0.000000,0.000000
5,2022-08-11 09:03:00,08:58,09:03,21302954,27.50,25.91,21302947,19.260,TP6336,47.216881,-122.473775,Sun,Y,0.099058,0.001732
6,2022-08-11 09:05:00,09:00,09:05,21302978,30.03,24.49,21223112,18.616,TP24408,47.216730,-122.479465,Shade,Y,0.146163,0.007437
7,2022-08-11 09:09:00,09:04,09:09,21302954,26.38,30.84,21302953,19.050,TP6335,47.220145,-122.473804,Sun,Y,0.062518,0.006099
8,2022-08-11 09:10:00,09:05,09:10,21302973,29.13,32.34,21223134,18.901,NO NUM,47.210787,-122.473819,Sun,Y,0.031416,0.000491
9,2022-08-11 09:11:00,09:06,09:11,21302978,25.74,32.52,21223132,19.472,TP14329,47.216757,-122.478987,Sun,Y,0.164934,0.004581
